# **Installing the necessary libraries**

In [ ]:
!pip -q install scikit-learn torch pandas datasets transformers

In [ ]:
import pandas as pd
from datasets import Dataset, load_dataset

# **Loading the data**

In [ ]:
print("Loading dataset...")
df = load_dataset("ailsntua/QEvasion")

def clarity_to_label(row) :
  mapping = {
      "Clear Reply": 0,
      "Ambivalent": 1,
      "Clear Non-Reply": 2
  }
  row["label"] = mapping[row["clarity_label"]]
  return row

df = df.map(clarity_to_label)
y_test = df["test"]["label"]




# **Defining the evaluation**

In [ ]:
def evaluate(y_test, y_test_pred):
    import matplotlib.pyplot as plt
    from sklearn.metrics import classification_report
    from sklearn.metrics import ConfusionMatrixDisplay
    from sklearn.metrics import precision_recall_fscore_support

    print(classification_report(y_test, y_test_pred))

    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test, y_test_pred, average="micro"
    )
    print(f"Precision: {precision:6.2f}")
    print(f"   Recall: {recall:6.2f}")
    print(f"       F1: {f1:6.2f}")

    ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred)
    plt.show()
    return precision, recall, f1



# **Loading the model and tokenizer**

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer



model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=3
)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


from transformers import pipeline

# we want to cut texts longer than 512 tokens to be 512 tokens
clf = pipeline(
    "text-classification",
    model=model,
    tokenizer=lambda x, **kwargs: tokenizer(
        x,
        padding="max_length",
        max_length=512,
        truncation=True,
        return_tensors="pt",
    ),
    device=0, # use GPU
)




# **Preparing the data for training**

In [ ]:
#tokenization, no longer than 512 tokens
tokenized_train = df["train"].map(
    lambda x: tokenizer(
        "Question: " + x["interview_question"] + " Answer: " + x["interview_answer"],
        padding="max_length",
        max_length=512,
        truncation=True
    ),
    batched=False
)

tokenized_test = df["test"].map(
    lambda x: tokenizer(
        "Question: " + x["interview_question"] + " Answer: " + x["interview_answer"],
        padding="max_length",
        max_length=512,
        truncation=True
    ),
    batched=False
)


# **Defining the evaluation function and training**

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


def compute_metrics(eval_pred):
    import numpy as np

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    p, r, f1 = evaluate(y_test, predictions)
    return {"precision": p, "recall": r, "f1": f1}



# train model
from transformers import Trainer, TrainingArguments

trainer = Trainer(
    model=model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
    args=TrainingArguments(
        num_train_epochs=3,
        eval_strategy="epoch",
        eval_steps=1,
        output_dir="output",
        seed=42,
        fp16=True,
    ),
)

trainer.train()